In [1]:
import urllib2, sys
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
from IPython.core.display import HTML
import requests
import time
import datetime
import urlparse
import os

In [2]:
class modifiedSoup(BeautifulSoup):
    def __init__(self, *args, **kwargs):
        self._url = None
        BeautifulSoup.__init__(self, *args, **kwargs)

In [3]:
def scrape(page_to_scrape,urls):
    matches = []
    for idx, url in enumerate(urls):
        matches.append(page_to_scrape(url))
        time.sleep(5)
        print'match {0} done'.format(idx)
    return matches

In [4]:
def scrape_match_data(team_name, startDate, endDate):
    teamID = get_teamID(team_name)
    params = {
        'teamID':teamID,
        'startDate':startDate,
        'endDate':endDate
    }

In [5]:
def get_teamID(team_name):
    team_info = pd.read_csv('../../scrapped_data/team_info.csv', index_col=0)
    teamID = team_info.loc[team_info.team_name == team_name, 'team_id'].tolist()[0]
    return teamID

In [6]:
def get_soup(url):
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = urllib2.Request(url,headers=hdr)
    page = urllib2.urlopen(req)
    soup = modifiedSoup(page, "lxml")
    soup._url = url
    print soup._url
    time.sleep(5)
    return soup

In [7]:
def get_matches_result_page_soup(params):
    match_page = "https://www.hltv.org/results?offset={offset}&content=demo&team={teamID}&startDate={" \
                 "startDate}&endDate={endDate}".format(**params)
    soup = get_soup(match_page)
    matches_soup = soup.find_all("div", class_='results-all')
    return matches_soup

In [8]:
def make_matches_url_loop(matches):
    urls = []
    results = matches[0].find_all("a", class_="a-reset")
    urls += ['https://www.hltv.org' + result['href'] for result in results]
    return urls

In [9]:
def make_matches_bof_loop(matches):
    bof = []
    results = matches[0].find_all("td", class_='result-score')
    bof += [result.text for result in results]
    return bof

In [10]:


def sum_digits_in_string(digit):
     return sum(int(x) for x in digit if x.isdigit())
        

In [11]:
def get_matches_result_page_urls_bof(params):
    done = False
    params['offset'] = 0

    while not done:
        matches = get_matches_result_page_soup(params)
        if len(matches) == 0:
            break

        urls = make_matches_url_loop(matches)
        bof = make_matches_bof_loop(matches)

        if len(urls) % 100 != 0:
            done = True
        else:
            params['offset'] += 100
    del params['offset']
    return {
        'urls' : urls,
        'bof' : bof
    }

In [16]:
def get_urls_from_columns(url):
    soup = get_soup(url)
    links = []   
    bof_site = soup.find_all("div", class_ ="columns")[0]
    for link in bof_site.find_all('a'):
        links.append("https://www.hltv.org" + str(link.get("href")))
    return links
        

In [21]:
def bof_testing(bof, url, type_of_parse):  # CANNOT BE USED WITH BASIC STATS PAGE PRIMARY_STATS_PAGE
   
    total_sum = sum_digits_in_string(bof)
    if total_sum > 15:
        return type_of_parse(url)
    all_matches = {}
    sites = get_urls_from_columns(url)
    if total_sum == 1:
        return 'forfeit'
    if total_sum >= 2:
        all_matches.update({'match_1': type_of_parse(sites[1]), 'match_2': type_of_parse(sites[2])})
        if total_sum >= 3:
            all_matches.update({'match_3': type_of_parse(sites[3])})
            if total_sum >= 4:
                all_matches.update({'match_4': type_of_parse(sites[4])})
                if total_sum == 5:
                    all_matches.update({'match_5': type_of_parse(sites[5])})
                else:
                        all_matches.update({'match_unknown': 'Unknown'})
    return all_matches
   

In [ ]:
def get_overview_data(url):
    soup = get_soup(url)
    match_time = soup.find_all("div", {"class": "small-text"})
    for item in match_time:
        match_time = item.text
    match_time = datetime.datetime.strptime(match_time, '%Y-%m-%d  %H:%MMap')  # match date and time


    round_history_team = soup.find_all("div", class_="round-history-team-row")  # winner of rounds and how rounds were won
    round_history_team_a = round_history_team[0].find_all("img")
    round_history_team_b = round_history_team[1].find_all("img")
    team_a_scores = []
    for scoreing in round_history_team_a:
        team_a_scores.append([scoreing.get('title')])  # rounds that team a won
    team_b_scores = []
    for scoreing in round_history_team_b:
        team_b_scores.append([scoreing.get('title')])  # rounds that team b won
    team_a_ending = []
    for ending in round_history_team_a:
        url = urlparse.urlparse(ending.get('src'))
        base = os.path.basename(url.path)  # how team a won the round
        team_a_ending.append([os.path.splitext(base)[0]])
    team_b_ending = []
    for ending in round_history_team_b:
        url = urlparse.urlparse(ending.get('src'))
        base = os.path.basename(url.path)
        team_b_ending.append([os.path.splitext(base)[0]])  # how team b won the round
    return {
        'match_time': match_time,  # match date and time
        'team_scores': [team_a_scores, team_b_scores],  # rounds that team a won
        'team_endings': [team_a_ending, team_b_ending]  # how the team won the round
    }


In [ ]:
def get_performance_data(url):
    tables = get_tables(url)
    total_team_kda = tables[0]
    who_kill_who = tables[1]
    first_kills = tables[2]
    awp_kills = tables[3]
    return {
        'total_team_kda': total_team_kda, # total kills deaths and assists of team
        'who_kill_who' : who_kill_who, # who killed who
        'first_kills' : first_kills, #first kill of the round
        'awp_kills' : awp_kills #awp kills
    }

In [19]:
def get_heatmap_data(url):
    soup = get_soup(url)
    divs = soup.find_all('div', class_='heatmap heatmap-data')
    heatmap_1 = divs[0]['data-heatmap-config']
    heatmap_2 = divs[1]['data-heatmap-config']
    return{
            "heat_maps" : [heatmap_1, heatmap_2]
        }